# Extract Respiration Signal with PhysFormer

In [ ]:
import torch
import respiration.utils as utils
from respiration.dataset import VitalCamSet
from respiration.extractor.rhythm_former import *

dataset = VitalCamSet()
natural_lighting = dataset.get_scenarios(['101_natural_lighting'])

# The pre-trained PPG models
models = {
    'MMPD_intra_RhythmFormer': {
        'model_path': utils.file_path('data', 'rhythm_former', 'MMPD_intra_RhythmFormer.pth'),
        'testing_scenarios': natural_lighting,
        'input_dimension': (128, 128),
    }, 'PURE_cross_RhythmFormer': {
        'model_path': utils.file_path('data', 'rhythm_former', 'PURE_cross_RhythmFormer.pth'),
        'testing_scenarios': natural_lighting,
        'input_dimension': (128, 128),
    }, 'UBFC_cross_RhythmFormer': {
        'model_path': utils.file_path('data', 'rhythm_former', 'UBFC_cross_RhythmFormer.pth'),
        'testing_scenarios': natural_lighting,
        'input_dimension': (128, 128),
    }
}

In [ ]:
# The fine-tuned Respiratory models
fine_tuned_ids = [
    # Version 1
    # '20240726_104536',
    # '20240726_155024',
    # '20240726_171101',
    # '20240726_212436',
    # '20240727_115407',
    # '20240727_170156',
    # '20240731_113403',
    # '20240801_124757',
    # '20240801_195728',
    # '20240802_155121',
    # '20240803_164403',
    # '20240804_191911',
    # '20240805_104628',
    # '20240805_200748',
    # '20240809_162808',
    # '20240809_234509',
    # '20240812_153436',
    # '20240812_204742',
    # '20240813_101414',
    # '20240816_205118',
    # '20240817_113146',
    # # '20240805_152219',
    # '20240819_214626',
    # '20240820_102249',
    # '20240820_155933',
    # '20240820_185602',
    # '20240831_115130',
    # '20240831_160651',

    # Version 2
    '20240902_094648',
    '20240902_123124',
    '20240902_152753',
    '20240902_181443',
    '20240902_210159',
    '20240902_234749',
    '20240903_023334',
    '20240903_051739',
    '20240903_080307',
    '20240903_104800',
    '20240903_171319',
    '20240903_230906',
    '20240904_001421',
    '20240904_011940',
    '20240904_022517',
    '20240904_231209',
    '20240905_094727',
    '20240905_134835',
    '20240905_163112',
    '20240905_173644',
    '20240905_233757',
    '20240906_022111',
    '20240906_213402',
    '20240907_001728',
    '20240907_190942',
    '20240907_215246',
    '20240908_194112',
]

for model_id in fine_tuned_ids:
    model_dir = utils.dir_path('models', 'rhythm_former_v2', model_id)
    manifest_path = utils.file_path(model_dir, 'manifest.json')
    manifest = utils.read_json(manifest_path)

    model_name = f'RF_{model_id}'
    model_path = manifest['models'][-1]['model_file']
    testing = manifest['testing_scenarios']
    input_dimension = manifest['size'] if 'size' in manifest else (128, 128)

    models[model_name] = {
        'model_path': model_path,
        'testing_scenarios': testing,
        'input_dimension': input_dimension,
    }

In [ ]:
# Create a list of subjects and settings that match to model's
scenarios = {}

for model_name, spec in models.items():
    for scenario in spec['testing_scenarios']:
        name = '-'.join(scenario)
        if name not in scenarios:
            scenarios[name] = [model_name]
        else:
            scenarios[name].append(model_name)

scenarios

In [ ]:
from tqdm.auto import tqdm
from datetime import datetime

device = utils.get_torch_device()
predictions = []

for scenario, model_names in tqdm(scenarios.items(), total=len(scenarios)):
    print(f'Extracting signal for {scenario}')

    subject, setting = scenario.split('-')
    frames, params = dataset.get_video_rgb(subject, setting)
    frames = utils.preprocess_frames(frames, (128, 128))

    for model_name in model_names:
        print(f'Using model: {model_name}')
        model_path = models[model_name]['model_path']

        model = RhythmFormer()
        # Fix model loading: Some key have an extra 'module.' prefix
        model = torch.nn.DataParallel(model)
        model.to(device)

        # Load the model
        _ = model.load_state_dict(torch.load(model_path, map_location=device))

        outputs = []

        start_time = datetime.now()

        with torch.no_grad():
            model.eval()
            output = model(frames)
            outputs.extend(output.squeeze().cpu().numpy().tolist())

        predictions.append({
            'subject': subject,
            'setting': setting,
            'model': model_name,
            'time': datetime.now() - start_time,
            'signal': outputs,
        })

    del frames, params

In [ ]:
import pandas as pd
import respiration.utils as utils

signal_dir = utils.dir_path('outputs', 'signals', mkdir=True)
signal_file = utils.file_path(signal_dir, 'rhythm_former.csv')

df = pd.DataFrame(predictions)
df.to_csv(signal_file, index=False)
df.head()